# Joint Undergraduate Researcher Onboarding #3
**Topic:** Analysis in pymatgen for 1) structure matching, 2) local environment coordinator, 3) site volume

**Date:** March 28, 2022

**Prepared by:** Ann Rutt

# Outline & Relevant Documentation

Onboarding Session Demonstration:
* Get structures for olivine LiFePO4 for demonstration
* `StructureMatcher()`: for comparing and grouping by structure
* `CrystalNN()`: for analyzing coordination based on neighest neighbor algorithm
* `ChemEnv` simplest vs. multi weights strategy: advanced method for analyzing coordination
* `VoronoiNN()`: for getting site specific volume

Onboarding Independent Exercises Goals:
* Reinforce how to work with `MPRester()` and manipulate pymatgen's structure objects
* Practice using structure grouping and comparison methods from `StructureMatcher()`
* Practice using `get_cn()` and `get_local_order_parameters()` methodes from `CrystalNN()`
* Practice getting site volumes using `VoronoiNN()`
* Exercise materials were selected to provide **context for factors that influence solid-state mobility** such as how anion selection and coordination can impact the working ion site volume.

For further reading and general reference...
* Pymatgen: https://github.com/materialsproject/pymatgen
* Structure Matcher: https://github.com/materialsproject/pymatgen/blob/master/pymatgen/analysis/structure_matcher.py#L298
* CrystalNN: https://dx.doi.org/10.1021/acs.inorgchem.0c02996
* ChemEnv: https://doi.org/10.1021/acs.chemmater.7b02766
* Site Volume via VoronoiNN: https://github.com/materialsproject/pymatgen/blob/master/pymatgen/analysis/local_env.py#L614

# Onboarding Session Demonstration

In [1]:
from pymatgen.core import Structure, Element, Lattice, PeriodicSite
from pymatgen.ext.matproj import MPRester

/Users/tashalewis/opt/anaconda3/envs/cms/lib/python3.9/site-packages/pymatgen/analysis/phase_diagram.py:26: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


## Get Structures

In [2]:
mpr = MPRester()

/Users/tashalewis/opt/anaconda3/envs/cms/lib/python3.9/site-packages/mp_api/client/client.py:139: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn("mpcontribs-client not installed. "


In [3]:
s1 = mpr.get_structure_by_material_id("mp-20361") # empty olivine FePO4
s2 = mpr.get_structure_by_material_id("mp-19017") # lithiated olivine FePO4

Retrieving MaterialsDoc documents: 100%|██████████| 1/1 [00:00<00:00, 15196.75it/s]


In [4]:
print(s1.composition)
print(s2.composition)

Fe4 P4 O16
Li4 Fe4 P4 O16


### Structure Matcher

In [5]:
from pymatgen.analysis.structure_matcher import StructureMatcher

In [6]:
sm = StructureMatcher()

In [7]:
sm.fit(s1,s1)

True

In [8]:
# will return None if provided structures do not have the same number of sites, False only returned if number of sites match
sm.fit(s1,s2)

In [9]:
sm_woli = StructureMatcher(ignored_species=[Element("Li")])

In [10]:
sm_woli.fit(s1,s2)

True

In [11]:
grouped_structs = sm.group_structures([s1,s1,s2,s1])
print(len(grouped_structs))

2


### CrystalNN

Analyzes the coordination for a single site specified by the user

In [12]:
from pymatgen.analysis.local_env import CrystalNN

In [13]:
cnn = CrystalNN()

In [14]:
cnn.get_cn(s2,0)

/global/homes/a/acrutt/.conda/envs/cms/lib/python3.8/site-packages/pymatgen/analysis/local_env.py:4137: UserWarning: No oxidation states specified on sites! For better results, set the site oxidation states in the structure.
  warnings.warn(
/global/homes/a/acrutt/.conda/envs/cms/lib/python3.8/site-packages/pymatgen/analysis/local_env.py:3940: UserWarning: CrystalNN: cannot locate an appropriate radius, covalent or atomic radii will be used, this can lead to non-optimal results.
  warnings.warn(


6

In [15]:
cnn.get_local_order_parameters(s2,0)

{'hexagonal planar': 0.164617403761537,
 'octahedral': 0.743883891668781,
 'pentagonal pyramidal': 0.37622166064529555}

In [16]:
cnn.get_nn(s2,0)

[PeriodicSite: O (-0.4482, 1.5226, 1.3909) [0.2930, -0.0429, 0.2500],
 PeriodicSite: O (0.4484, -1.5225, -1.3915) [-0.2932, 0.0429, -0.2500],
 PeriodicSite: O (1.0103, 1.5226, -1.2252) [-0.2581, 0.0967, 0.2500],
 PeriodicSite: O (-1.0106, -1.5226, 1.2249) [0.2581, -0.0968, -0.2500],
 PeriodicSite: O (-1.7281, -0.2865, -1.3504) [-0.2845, -0.1655, -0.0470],
 PeriodicSite: O (1.7284, 0.2865, 1.3504) [0.2845, 0.1655, 0.0470]]

### ChemEnv
Analyzes the coordination for all sites corresponding to the specified element

In [17]:
from pymatgen.analysis.chemenv.coordination_environments.coordination_geometry_finder import LocalGeometryFinder
from pymatgen.analysis.chemenv.coordination_environments.chemenv_strategies import SimplestChemenvStrategy, MultiWeightsChemenvStrategy
from pymatgen.analysis.chemenv.coordination_environments.structure_environments import LightStructureEnvironments

In [18]:
lgf = LocalGeometryFinder()


If you use the ChemEnv tool for your research, please consider citing the following reference(s) :
David Waroquiers, Xavier Gonze, Gian-Marco Rignanese, Cathrin Welker-Nieuwoudt, Frank Rosowski,
Michael Goebel, Stephan Schenk, Peter Degelmann, Rute Andre, Robert Glaum, and Geoffroy Hautier,
"Statistical analysis of coordination environments in oxides",
Chem. Mater., 2017, 29 (19), pp 8346-8360,
DOI: 10.1021/acs.chemmater.7b02766



In [19]:
lgf.setup_structure(structure=s2)

In [20]:
se = lgf.compute_structure_environments(maximum_distance_factor=1.5, 
                                        only_atoms=["Li"], # must identify specie interested in
                                        max_cn=8, 
                                        min_cn=2, 
                                        minimum_angle_factor=0.05)

In [21]:
strategy = SimplestChemenvStrategy(distance_cutoff=1.5, angle_cutoff=0.3)
lse = LightStructureEnvironments.from_structure_environments(strategy=strategy, structure_environments=se)
lse.coordination_environments

[[{'ce_symbol': 'O:6',
   'ce_fraction': 1.0,
   'csm': 2.1010416039777606,
   'permutation': [0, 5, 4, 1, 3, 2]}],
 [{'ce_symbol': 'O:6',
   'ce_fraction': 1.0,
   'csm': 2.1012978449464805,
   'permutation': [1, 4, 5, 0, 3, 2]}],
 [{'ce_symbol': 'O:6',
   'ce_fraction': 1.0,
   'csm': 2.1010210124128483,
   'permutation': [5, 4, 3, 2, 1, 0]}],
 [{'ce_symbol': 'O:6',
   'ce_fraction': 1.0,
   'csm': 2.1010261009944227,
   'permutation': [4, 2, 3, 1, 0, 5]}],
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [22]:
multi_strategy = MultiWeightsChemenvStrategy.stats_article_weights_parameters()
multi_lse = LightStructureEnvironments.from_structure_environments(strategy=multi_strategy, structure_environments=se)
multi_lse.coordination_environments

[[{'ce_symbol': 'O:6',
   'ce_fraction': 1.0,
   'csm': 2.1010416039777606,
   'permutation': [0, 5, 4, 1, 3, 2]}],
 [{'ce_symbol': 'O:6',
   'ce_fraction': 1.0,
   'csm': 2.1012978449464805,
   'permutation': [1, 4, 5, 0, 3, 2]}],
 [{'ce_symbol': 'O:6',
   'ce_fraction': 1.0,
   'csm': 2.1010210124128483,
   'permutation': [5, 4, 3, 2, 1, 0]}],
 [{'ce_symbol': 'O:6',
   'ce_fraction': 1.0,
   'csm': 2.1010261009944227,
   'permutation': [4, 2, 3, 1, 0, 5]}],
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

### Site Volume

In [23]:
from pymatgen.analysis.local_env import VoronoiNN

In [24]:
vnn = VoronoiNN()

In [25]:
for n,d in vnn.get_voronoi_polyhedra(s2,0).items():
    print(n,d["volume"])

20 0.09969843080414519
17 0.031785553911123754
9 0.15887804556992208
3 1.7062376245790218
7 0.016473853236948638
4 1.706434011825301
1 1.6848895746369061
19 0.09957777968030611
6 1.7287521232581515
10 0.1591181332141011
2 1.6851011882198987
18 0.0317683658142357
5 1.728243940455113
8 0.016280256693184354


In [26]:
# sum to get total volume associated with specified site
sum([d["volume"] for d in vnn.get_voronoi_polyhedra(s2,0).values()])

10.853238881898358

# Onboarding Independent Exercises

Use the following 5 structures for these exercises: `mp-25275`, `mp-22584`, `mp-677217`, `mp-1045420`, `mp-1443666`

1. Identify the working ion in each material (e.g. Li, Na, Mg, Ca, etc.)

2. How many distict structures (including working ions) are present? What are the mp_ids of any materials with the same structure (if any)? (hint: try the `anonymous` key word argument or `fit_anonymous` method of structure matcher)

3. How many distict host structures (ignoring working ions) are there?

4. Using CrystalNN, what is the most likely local order parameter for the working ion sites in `mp-1045420` and `mp-1443666`?

5. Using CrystalNN how does the coordination of the working ion compare between `mp-22584` and `mp-1443666`?

6. How does the working ion site volume compare between `mp-22584` and `mp-1443666`?

7. How does the working ion site volume compare between `mp-1045420` and `mp-1443666`?

8. Pick one structure to verify that the total volume of the structure matches the sum of the individual site volumes

**Bonus:** What insights can we draw from our results from answering #4-7?

**Structure/mp_id Key:**

* olivine FePO4: mp-20361
* Li in olivine FePO4: mp-19017
* spinel MnO2: mp-25275
* Li in spinel Mn2O4: mp-22584
* Mg in chevrel Mo6S8: mp-677217
* oct Mg in spinel Ti2S4: mp-1045420
* tet Mg in spinel Ti2S4: mp-1443666
* hollandite MnO2: mp-19395
* birnessite MnO2: mp-25424

### 1. Identify the working ion in each material (e.g. Li, Na, Mg, Ca, etc.)

In [27]:
mp_ids = ["mp-25275", "mp-22584", "mp-677217", "mp-1045420", "mp-1443666"]
structs = []
for mp_id in mp_ids:
    structs.append(mpr.get_structure_by_material_id(mp_id))

/global/homes/a/acrutt/.conda/envs/cms/lib/python3.8/site-packages/pymatgen/ext/matproj.py:685: UserWarning: The calculation task mp-1045420 is mapped to canonical mp-id mvc-16768, so structure for mvc-16768 returned. This is not an error, see documentation. If original task data for mp-1045420 is required, use get_task_data(). To find the canonical mp-id from a task id use get_materials_id_from_task_id().
  warnings.warn(
/global/homes/a/acrutt/.conda/envs/cms/lib/python3.8/site-packages/pymatgen/ext/matproj.py:685: UserWarning: The calculation task mp-1443666 is mapped to canonical mp-id mvc-27, so structure for mvc-27 returned. This is not an error, see documentation. If original task data for mp-1443666 is required, use get_task_data(). To find the canonical mp-id from a task id use get_materials_id_from_task_id().
  warnings.warn(


In [28]:
for mp_id,struct in zip(mp_ids,structs):
    print(mp_id,struct.composition)

mp-25275 Mn4 O8
mp-22584 Li2 Mn4 O8
mp-677217 Mg2 Mo6 S8
mp-1045420 Mg2 Ti4 S8
mp-1443666 Mg2 Ti4 S8


### 2. How many distict structures (including working ions) are present? What are the mp_ids of any materials with the same structure (if any)? (hint: try the `anonymous` key word argument or `fit_anonymous` method of structure matcher)

In [29]:
grouped_structs = sm.group_structures(structs,anonymous=True)
print(len(grouped_structs))

4


In [30]:
for group in grouped_structs:
    for mp_id,struct in zip(mp_ids,structs):
        if sm.fit_anonymous(group[0],struct):
            print(mp_id)
    print()

mp-25275

mp-22584
mp-1443666

mp-1045420

mp-677217



### 3. How many distict host structures (ignoring working ions) are there?

In [31]:
sm_wowi = StructureMatcher(ignored_species=[Element("Li"),Element("Mg")])

In [32]:
print(len(sm_wowi.group_structures(structs,anonymous=True)))

2


### 4. Using CrystalNN, what is the most likely local order parameter for the working ion sites in `mp-1045420` and `mp-1443666`?

In [33]:
from pprint import pprint

In [34]:
for mp_id,struct in zip(mp_ids,structs):
    if mp_id in ["mp-1045420","mp-1443666"]:
        print(mp_id,struct)
        d = cnn.get_local_order_parameters(struct,0) # just print the first site in structure
        pprint(d)
        print()

mp-1045420 Full Formula (Mg2 Ti4 S8)
Reduced Formula: Mg(TiS2)2
abc   :   7.089700   7.089700   7.089700
angles: 119.626584 119.588733  90.680912
Sites (14)
  #  SP           a         b         c    magmom
---  ----  --------  --------  --------  --------
  0  Mg    0.5       0.5       0           -0
  1  Mg    0         0.5       0.5         -0
  2  Ti    0         0.5       0           -0.005
  3  Ti    0.5       0         0.5         -0.006
  4  Ti    0         0         0.5         -0.005
  5  Ti    0         0         0           -0.006
  6  S     0.760971  0.753127  0.007844     0
  7  S     0.250514  0.756215  0.0057      -0
  8  S     0.245282  0.753127  0.492156     0
  9  S     0.749486  0.755186  0.5057      -0
 10  S     0.754718  0.246873  0.507844     0
 11  S     0.749486  0.243785  0.9943      -0
 12  S     0.250514  0.244814  0.4943      -0
 13  S     0.239029  0.246873  0.992156     0
{'hexagonal planar': 0.0001733216199670461,
 'octahedral': 0.9960356244346299,
 'pe

In [35]:
for mp_id,struct in zip(mp_ids,structs):
    if mp_id in ["mp-1045420","mp-1443666"]:
        print(mp_id,struct.composition)
        for n,s in enumerate(struct):
            if str(s.specie) in ["Li","Mg"]:
                d = cnn.get_local_order_parameters(struct,n)
                for k,v in d.items():
                    if v == max(d.values()):
                        print(k,v)
        print()

mp-1045420 Mg2 Ti4 S8
octahedral 0.9960356244346299
octahedral 0.9960350041234826

mp-1443666 Mg2 Ti4 S8
tetrahedral 1.0
tetrahedral 1.0



### 5. Using CrystalNN how does the coordination of the working ion compare between `mp-22584` and `mp-1443666`?

In [36]:
for mp_id,struct in zip(mp_ids,structs):
    if mp_id in ["mp-22584","mp-1443666"]:
        print(mp_id,struct.composition)
        for n,s in enumerate(struct):
            if str(s.specie) in ["Li","Mg"]:
                print(cnn.get_cn(struct,n),s)
        print()

mp-22584 Li2 Mn4 O8
4 [4.211237 4.211237 4.211237] Li
4 [6.3168555 6.3168555 6.3168555] Li

mp-1443666 Mg2 Ti4 S8
4 [5.179177 5.179177 5.179177] Mg
4 [7.7687655 7.7687655 7.7687655] Mg



### 6. How does the working ion site volume compare between `mp-22584` and `mp-1443666`?

In [37]:
for mp_id,struct in zip(mp_ids,structs):
    if mp_id in ["mp-22584","mp-1443666"]:
        print(mp_id,struct.composition)
        for n,s in enumerate(struct):
            if str(s.specie) in ["Li","Mg"]:
                print(sum([d["volume"] for d in vnn.get_voronoi_polyhedra(struct,n).values()]),s)
        print()

mp-22584 Li2 Mn4 O8
12.476338046309975 [4.211237 4.211237 4.211237] Li
12.476338046309973 [6.3168555 6.3168555 6.3168555] Li

mp-1443666 Mg2 Ti4 S8
21.69680035777642 [5.179177 5.179177 5.179177] Mg
21.696800357776436 [7.7687655 7.7687655 7.7687655] Mg



### 7. How does the working ion site volume compare between `mp-1045420` and `mp-1443666`?

In [38]:
for mp_id,struct in zip(mp_ids,structs):
    if mp_id in ["mp-1045420","mp-1443666"]:
        print(mp_id,struct.composition)
        for n,s in enumerate(struct):
            if str(s.specie) in ["Li","Mg"]:
                print(sum([d["volume"] for d in vnn.get_voronoi_polyhedra(struct,n).values()]),s)
        print()

mp-1045420 Mg2 Ti4 S8
16.41108952850107 [0.       0.       4.983298] Mg
16.411027409606447 [3.564839 0.       0.      ] Mg

mp-1443666 Mg2 Ti4 S8
21.69680035777642 [5.179177 5.179177 5.179177] Mg
21.696800357776436 [7.7687655 7.7687655 7.7687655] Mg



### 8. Pick one structure to verify that the total volume of the structure matches the sum of the individual site volumes

In [39]:
struct = structs[0]
print(struct.volume)

139.8945617779943


In [40]:
vol = 0
for n,s in enumerate(struct):
    vol += sum([d["volume"] for d in vnn.get_voronoi_polyhedra(struct,n).values()])
print(vol)

139.89456177799434


### **Bonus:** What insights can we draw from our results from answering #4-7?